In [19]:
import pandas as pd
import nltk
# nltk.download('all')
import re
import numpy as np
import collections
import sklearn as sk
import string
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

In [20]:
df = pd.read_csv("../B_Data_Cleaning/synthetic_reviews_all.csv")
df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
df["Review"] = df["Review"].astype("str")
df

,Review,Recommended or Not Recommended
0,unnamed holophone gluten fundamentals dividen ...,True
1,thoughtthrough chargeor gift prepurchased upat...,True
2,reopening sequences boba ультранизкая leaked f...,True
3,domingo skimming esse rollercoaster etkili ski...,True
4,chooseyourownadventure solitary een stairs yee...,True
...,...,...
16455,bigbudget ух railways definition wooooaaa unin...,False
16456,horror freakishly kurguluyor chunky cobwebs bc...,False
16457,sinful abstracted attentionon sunlight brecomm...,False
16458,charactercar daemons lt cruising ахуенно slews...,False


In [21]:
# Checking the length of all the reviews
(df["Review"]=='').value_counts()

False    16460
Name: Review, dtype: int64

In [22]:
# make a set_column to count the number of words in each review
df["set_column"] = df["Review"].apply(lambda x: set(x.split()))
df

,Review,Recommended or Not Recommended,set_column
0,unnamed holophone gluten fundamentals dividen ...,True,"{atlast, lingo, noti, beach, itinyi, disgraced..."
1,thoughtthrough chargeor gift prepurchased upat...,True,"{tennis, gunfight, goodness, coats, customize,..."
2,reopening sequences boba ультранизкая leaked f...,True,"{mightve, fluro, hyped, jogar, satupid, realiz..."
3,domingo skimming esse rollercoaster etkili ski...,True,"{libertarianright, concluídos, chart, acaba, p..."
4,chooseyourownadventure solitary een stairs yee...,True,"{ehi, hulahand, justly, verhicles, buggery, em..."
...,...,...,...
16455,bigbudget ух railways definition wooooaaa unin...,False,"{fabricar, opposite, завязка, reconstructions,..."
16456,horror freakishly kurguluyor chunky cobwebs bc...,False,"{massively, eventgig, recurring, berserker, ma..."
16457,sinful abstracted attentionon sunlight brecomm...,False,"{exploitation, wizzy, أسلوب, modifications, ma..."
16458,charactercar daemons lt cruising ахуенно slews...,False,"{lt, hmechanicsh, blergh, играм, rdrgta, tomor..."


In [23]:
# split the data into train and test
train, test = sk.model_selection.train_test_split(df, test_size=0.33, random_state=42)

In [24]:
# Dividing dataset into recommended and not recommended
df_recom = train.loc[train["Recommended or Not Recommended"] == True,:]
df_recom = df_recom.reset_index(drop=True)

df_not_recom = train.loc[train["Recommended or Not Recommended"] == False,:]
df_not_recom = df_not_recom.reset_index(drop=True)

In [25]:
# This is to make the set dictionary
# count how many times each word appears in the reviews
bow_recom_temp = collections.Counter([y for x in df_recom.set_column for y in x])
bow_not_recom_temp = collections.Counter([y for x in df_not_recom.set_column for y in x])

bow_recom_set = dict(bow_recom_temp)
bow_not_recom_set = dict(bow_not_recom_temp)

for key in bow_recom_set:
    if key not in bow_not_recom_set:
        bow_not_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_set:
    if key not in bow_recom_set:
        bow_recom_set[key] = 0
        pass
    else:
        pass

for key in bow_recom_set:
    bow_recom_set[key] += 1

for key in bow_not_recom_set:
    bow_not_recom_set[key] += 1


In [26]:
# This is to make the list dictionary
# count how many times each word appears in the reviews
bow_recom = collections.Counter([y for x in df_recom.Review for y in x.split()])
bow_not_recom = collections.Counter([y for x in df_not_recom.Review for y in x.split()])

bow_recom_dict = dict(bow_recom)
bow_not_recom_dict = dict(bow_not_recom)

for key in bow_recom_dict:
    if key not in bow_not_recom_dict:
        bow_not_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_not_recom_dict:
    if key not in bow_recom_dict:
        bow_recom_dict[key] = 0
        pass
    else:
        pass

for key in bow_recom_dict:
    bow_recom_dict[key] += 1

for key in bow_not_recom_dict:
    bow_not_recom_dict[key] += 1
    

In [27]:
bow_recom_sum_vals = sum(bow_recom_dict.values())
bow_not_recom_sum_vals = sum(bow_not_recom_dict.values())

for i in bow_recom_dict:

    bow_recom_dict[i] /= bow_recom_sum_vals

for i in bow_not_recom_dict:

    bow_not_recom_dict[i] /= bow_not_recom_sum_vals

In [28]:
assert len(bow_recom_dict) == len(bow_not_recom_dict)
assert len(bow_recom_set) == len(bow_not_recom_set)

In [29]:
test.drop(columns=["set_column"], inplace=True)

In [30]:
def define_sentiment_all(element,test1=False, test2=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                score *= prob_word_given_class*tf*idf
                # score *= prob_word_given_class*idf
                # score *= prob_word_given_class*tf
                # score *= prob_word_given_class

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    print(flag_count,i)
                    flag_count += 1


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [31]:
def define_sentiment_idf(element,test1=False, test2=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                # score *= prob_word_given_class*tf*idf
                score *= prob_word_given_class*idf
                # score *= prob_word_given_class*tf
                # score *= prob_word_given_class

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    print(flag_count,i)
                    flag_count += 1


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [32]:
def define_sentiment_tf(element,test1=False, test2=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                # score *= prob_word_given_class*tf*idf
                # score *= prob_word_given_class*idf
                score *= prob_word_given_class*tf
                # score *= prob_word_given_class

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    print(flag_count,i)
                    flag_count += 1


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [33]:
def define_sentiment_freq(element,test1=False, test2=False):

    """This function takes a review and returns the label for that review"""

    full_shape = df.shape[0]
    positive_review_probabiliy = len(df_recom)/full_shape
    negative_review_probabiliy = len(df_not_recom)/full_shape
    
    df_choices_positive = [positive_review_probabiliy, bow_recom_set, bow_recom_dict, df_recom]
    df_choices_negative = [negative_review_probabiliy ,bow_not_recom_set, bow_not_recom_dict, df_not_recom]

    def classifier(element, your_class = 'positive'):
        flag = {}
        flag_count = 0
        if your_class == 'positive':
            df_choices = df_choices_positive
        else:
            df_choices = df_choices_negative
        prob_of_class = df_choices[0]/full_shape
        score = 1 * prob_of_class
        score = float(format(score, '.12f'))
        # score = 0.1
        for i in element.split():
            if i not in df_choices[2].keys():
                pass
            else:
                prob_word_given_class = (df_choices[2])[i]
                prob_word_given_class = float(format(prob_word_given_class, '.12f'))
                # Almost the same value, given our spin on this application. 
                # Normally, this term frequency would be calculated differently across the positive and negative documents
                # but we are only looking at the reviews as the documents themselves to determine a word's relevance in the positive
                # or negative corpus. 
                tf = np.log(prob_word_given_class)
                # tf = float(format(tf, '.12f'))
                # tf = abs(np.log(prob_word_given_class))
                # The IDF is the number of reviews / the number of reviews that contain the word in that given corpus
                # idf = abs(np.log(df_choices[3].shape[0]/(df_choices[1])[i]))
                idf = np.log(df_choices[3].shape[0]/(df_choices[1])[i])
                # idf = float(format(idf, '.12f'))
                # score *= prob_word_given_class*tf*idf
                # score *= prob_word_given_class*idf
                # score *= prob_word_given_class*tf
                score *= prob_word_given_class

                if score < 0:
                    flag[flag_count] = (score, your_class)
                    print(flag_count,i)
                    flag_count += 1


        return score
    positive_score = classifier(element, 'positive')
    negative_score = classifier(element, 'negative')
    if positive_score > negative_score:
        return True
    elif positive_score == negative_score:
        # Choosing an arbitrary value, because we assume that a review with one or few words of little substance
        # is implied to be negative, as is usual with netizens. 
        return False
    else:
        return False

In [34]:
test["score_all"] = test.Review.apply(define_sentiment_all)
test["score_tf"] = test.Review.apply(define_sentiment_tf)
test["score_idf"] = test.Review.apply(define_sentiment_idf)
test["score_freq"] = test.Review.apply(define_sentiment_freq)

In [41]:
print(f"This is our accuracy for synthetic data when we use all the formula: {(sum(test['Recommended or Not Recommended'] == test['score_all'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for synthetic data when we use only frequency and TF: {(sum(test['Recommended or Not Recommended'] == test['score_tf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for synthetic data when we use only frequency and IDF: {(sum(test['Recommended or Not Recommended'] == test['score_idf'])/test.shape[0])*100:.2f}%")
print(f"This is our accuracy for synthetic data when we use only frequency: {(sum(test['Recommended or Not Recommended'] == test['score_freq'])/test.shape[0])*100:.2f}%")

This is our accuracy for synthetic data when we use all the formula: 45.88%
This is our accuracy for synthetic data when we use only frequency and TF: 46.72%
This is our accuracy for synthetic data when we use only frequency and IDF: 60.51%
This is our accuracy for synthetic data when we use only frequency: 55.41%


In [36]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_all'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_all']))


              precision    recall  f1-score   support

       False       0.33      0.59      0.42      1803
        True       0.66      0.39      0.49      3629

    accuracy                           0.46      5432
   macro avg       0.49      0.49      0.46      5432
weighted avg       0.55      0.46      0.47      5432



In [37]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_tf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_tf']))


              precision    recall  f1-score   support

       False       0.34      0.64      0.44      1803
        True       0.68      0.38      0.49      3629

    accuracy                           0.47      5432
   macro avg       0.51      0.51      0.47      5432
weighted avg       0.57      0.47      0.47      5432



In [38]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_idf'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_idf']))


              precision    recall  f1-score   support

       False       0.40      0.37      0.38      1803
        True       0.70      0.72      0.71      3629

    accuracy                           0.61      5432
   macro avg       0.55      0.55      0.55      5432
weighted avg       0.60      0.61      0.60      5432



In [39]:
# Generate a confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(test['Recommended or Not Recommended'], test['score_freq'])
# Generate a classification report
from sklearn.metrics import classification_report
print(classification_report(test['Recommended or Not Recommended'], test['score_freq']))


              precision    recall  f1-score   support

       False       0.38      0.52      0.44      1803
        True       0.70      0.57      0.63      3629

    accuracy                           0.55      5432
   macro avg       0.54      0.54      0.53      5432
weighted avg       0.60      0.55      0.57      5432

